# Практика к лекции 2, часть 1.
### База слов WordNet

Самое распространенное решение проблемы о передачи смысла слова компьютеру – использовать некую базу данных –, например, [**WordNet**](https://wordnet.princeton.edu/).

**WordNet** – это лексическая база данных английского языка, разработанная в Принстонском университете. Представляет собой электронный словарь и набор семантических сетей для английского языка.

Базовой единицей в свою очередь в WordNet является не отдельное слово, а синонимический ряд, объединяющий слова со схожим значением в узел семантической сети. Такие синонимические ряды называют синсетами – synset. Они связаны между собой различными семантическими отношениями:

- гипероним – обед -> прием пищи (lunch -> meal)
- гипоним – прием пищи -> обед (meal -> lunch)
- has-member – факультет -> профессор (faculty -> professor)
- member-of – пилот -> экипаж (pilot -> crew)
- мероним/has-part – стул -> ножка (chair -> leg)
- антоним – белое -> черное (white -> black)

In [1]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/maria/nltk_data...


True

In [2]:
from nltk.corpus import wordnet

Здесь мы составим вспомогательный dict, для вывода более красивого print 

Части речи в WordNet обозначаются одной буквой - 'n', 'v' etc 

Нам же чисто визуально удобнее смотреть на более полное название - сопоставим ключу 'n' в качестве значения слово 'noun' и так далее для всех остальных частей речи. 

In [3]:
poses = {'n': 'noun', 'v': 'verb', 's': 'adj (s)', 'a': 'adj', 'r': 'adv'}

Давайте выведем на экран все синсеты слова "good":

In [4]:
good = wordnet.synsets("good")
for synset in good: 
    print(f'{poses[synset.pos()]}: {", ".join([l.name() for l in synset.lemmas()])}')

noun: good
noun: good, goodness
noun: good, goodness
noun: commodity, trade_good, good
adj: good
adj (s): full, good
adj: good
adj (s): estimable, good, honorable, respectable
adj (s): beneficial, good
adj (s): good
adj (s): good, just, upright
adj (s): adept, expert, good, practiced, proficient, skillful, skilful
adj (s): good
adj (s): dear, good, near
adj (s): dependable, good, safe, secure
adj (s): good, right, ripe
adj (s): good, well
adj (s): effective, good, in_effect, in_force
adj (s): good
adj (s): good, serious
adj (s): good, sound
adj (s): good, salutary
adj (s): good, honest
adj (s): good, undecomposed, unspoiled, unspoilt
adj (s): good
adv: well, good
adv: thoroughly, soundly, good


Теперь найдемы **гиперонимы** к слову "elephant"

**Гипероним** - слово с более широким значением, выражающее общее, родовое понятие, название класса (множества) предметов (свойств, признаков).

[source](https://ru.wikipedia.org/wiki/%D0%93%D0%B8%D0%BF%D0%BE%D0%BD%D0%B8%D0%BC_%D0%B8_%D0%B3%D0%B8%D0%BF%D0%B5%D1%80%D0%BE%D0%BD%D0%B8%D0%BC)

In [5]:
elephant = wordnet.synset("elephant.n.01")
hypernym = lambda s: s.hypernyms()
list(elephant.closure(hypernym))

[Synset('pachyderm.n.01'),
 Synset('proboscidean.n.01'),
 Synset('placental.n.01'),
 Synset('placental.n.01'),
 Synset('mammal.n.01'),
 Synset('mammal.n.01'),
 Synset('vertebrate.n.01'),
 Synset('vertebrate.n.01'),
 Synset('chordate.n.01'),
 Synset('chordate.n.01'),
 Synset('animal.n.01'),
 Synset('animal.n.01'),
 Synset('organism.n.01'),
 Synset('organism.n.01'),
 Synset('living_thing.n.01'),
 Synset('living_thing.n.01'),
 Synset('whole.n.02'),
 Synset('whole.n.02'),
 Synset('object.n.01'),
 Synset('object.n.01'),
 Synset('physical_entity.n.01'),
 Synset('physical_entity.n.01'),
 Synset('entity.n.01'),
 Synset('entity.n.01')]

Проблемы использования **WordNet**:
- это отличный ресурс, в котором очень много данных, однако он опускает некоторые нюансы, на которые отличается смысл слов. 

Например, *“good”* – «хороший» иp *“proficient”* – «опытный», «искусный». Эти слова будут находиться вместе в одном из синсетов для слова *“good”*, т.е. согласно WordNet иметь схожий смысл, однако, как даже видно по переводу, они все же имеют разный смысл.
- здесь отсутствует смысл достаточно новых слов –*”stan”* (новое слово в английском языке, означает “невероятно увлеченный и преданный фанат” - “stalker-fan”),  *“badass”* – WordNet невероятно сложно поддерживать в актуальном состоянии, так как его необходимо обновлять вручную и нанимать дорогостоящих специалистов — такие вещи обычно не доверяют случайным толокерам.

- сложно вычислить точную «близость» слов друг к другу по смыслу. 

Так как WordNet по сути состоит из неизменяемых дискретных наборов связей между словами, так что если у нас есть слово, которое близко по значению к заданному, но отсутствует в синсете для заданного слова, мы не сможем никак узнать, что они похожи по смыслу и использовать это. Например слова *“good”* и *“marvelous”* похожи по значению в разных контекстах, однако мы не узнаем об этом, используя WordNet, т.к. они отсутствуют друг у друга в синсетах


Пример определения близости слов с помощью одного из методов WordNet - **path_similarity**

**path_similarity** возвращает "оценку" близости - число от 0 до 1 - на основе кратчайшего пути, который соединяет слова в таксономии is-a (гипероним/гипноим)

In [6]:
cat = wordnet.synset("cat.n.01")
dog = wordnet.synset("dog.n.01")
kitten = wordnet.synset("kitten.n.01")
print(cat.path_similarity(cat))
print(cat.path_similarity(dog))
print(cat.path_similarity(kitten))

1.0
0.2
0.09090909090909091


Работает не очень, правда? :)

Mожно вывести на экран определение слова:

In [7]:
gradient = wordnet.synset("gradient.n.01")
gradient.definition()

'a graded change in the magnitude of some physical quantity or dimension'

Попробуйте сами вывести, например, антонимы к слову "wicked" с помощью метода antonyms()